In [1]:
import great_expectations as gx

In [2]:
gx.__version__

'1.0.5'

In [3]:
context = gx.get_context(mode="file")
print(type(context))

<class 'great_expectations.data_context.data_context.file_data_context.FileDataContext'>


In [4]:
source_folder = "./data/first_ten_trips_in_each_file/"
data_source_name = "taxi_first_ten_trips"

In [5]:
data_source = context.data_sources.add_pandas_filesystem(
    name=data_source_name, base_directory=source_folder
)

In [6]:
asset_name = "taxi_first_ten_trips_asset"

In [7]:
file_csv_asset = data_source.add_csv_asset(name=asset_name)

In [8]:
batch_definition_name = "monthly_yellow_tripdata_sample"

In [13]:
batch_definition_regex = (
    r"yellow_tripdata_sample_(?P<year>\d{4})-(?P<month>\d{2})\.csv"
)

In [20]:
file_csv_asset.batch_definitions

[BatchDefinition(id=UUID('1218768a-c221-4e42-b142-af89a0b857ea'), name='monthly_yellow_tripdata_sample', partitioner=FileNamePartitionerYearly(regex=re.compile('yellow_tripdata_sample_(?P<year>\\d{4})-(?P<month>\\d{2})\\.csv'), sort_ascending=True, param_names=['year', 'month']))]

In [17]:
# file_csv_asset.delete_batch_definition(name=batch_definition_name)

In [18]:
batch_definition = file_csv_asset.add_batch_definition_monthly(
    name=batch_definition_name, regex=batch_definition_regex
)

In [19]:
# Verify setup
batch = batch_definition.get_batch(batch_parameters={"year": "2019", "month": "01"})
batch.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

   vendor_id      pickup_datetime     dropoff_datetime  passenger_count  \
0          1  2019-01-15 03:36:12  2019-01-15 03:42:19                1   
1          1  2019-01-25 18:20:32  2019-01-25 18:26:55                1   
2          1  2019-01-05 06:47:31  2019-01-05 06:52:19                1   
3          1  2019-01-09 15:08:02  2019-01-09 15:20:17                1   
4          1  2019-01-25 18:49:51  2019-01-25 18:56:44                1   

   trip_distance  rate_code_id store_and_fwd_flag  pickup_location_id  \
0            1.0             1                  N                 230   
1            0.8             1                  N                 112   
2            1.1             1                  N                 107   
3            2.5             1                  N                 143   
4            0.8             1                  N                 246   

   dropoff_location_id  payment_type  fare_amount  extra  mta_tax  tip_amount  \
0                   48       

In [21]:
import great_expectations.expectations as gxe

In [24]:
between = gxe.ExpectColumnValuesToBeBetween(column="passenger_count", min_value=0, max_value=6)

In [25]:
batch.validate(between)

Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_counts": [],
    "partial_unexpected_index_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}